# Predictive Modelling with Regression

In [ ]:
from datetime import datetime
import pandas as pd
from scripts.helper import reduce_mem_usage
from pycaret.regression import *
from pycaret.utils import check_metric
from pycaret.datasets import get_data
import pickle

## Import Libraries

In [ ]:
# Create datetime
today = datetime.today()
d1 = today.strftime("%d%m%Y")

## Import Datasets

In [ ]:
# Test dataset from pycaret library for classification modelling
#dataset_pycaret = get_data('credit')

# Capstone dataset!
dataset = pd.read_csv('data/feat_train_v2.csv')
dataset_test = pd.read_csv('data/feat_test_v2.csv')

In [ ]:
print("There are {} observations and {} features in this featured train dataset. \n".format(dataset.shape[0],dataset.shape[1]))

In [ ]:
print("There are {} observations and {} features in this featured test dataset. \n".format(dataset_test.shape[0],dataset_test.shape[1]))

In [ ]:
#dataset_pycaret.info()

## Preparation for Modelling

In [ ]:
numerical_cols = np.load("data/Numerical_Columns.npy")
categorical_cols = np.load("data/Categorical_Columns.npy")

In [ ]:
type(numerical_cols)

In [ ]:
numerical_cols = numerical_cols.tolist()
categorical_cols = categorical_cols.tolist()

In [ ]:
type(numerical_cols)

In [ ]:
test_id = dataset_test['fullVisitorId'].values

## Create Targets and Features

In [ ]:
# Create target for classification model
reg_train = dataset[categorical_cols+numerical_cols]
reg_train['Target'] = dataset['totals.transactionRevenue']
reg_test = dataset_test[categorical_cols+numerical_cols]
reg_test['Target'] = dataset_test['totals.transactionRevenue']

In [ ]:
totals_transactionRevenue_zero = reg_train[reg_train['Target'] == 0].sample(frac=1, random_state=10)
totals_transactionRevenue_nonzero = reg_train[reg_train['Target'] != 0]
reg_train = pd.concat([totals_transactionRevenue_zero, totals_transactionRevenue_nonzero], axis=0)

In [ ]:
reg_train["Target"] = reg_train["Target"].apply(np.log1p)
reg_test["Target"] = reg_test["Target"].apply(np.log1p)

In [ ]:
print("There are {} observations and {} features in this train dataset. \n".format(reg_train.shape[0],reg_train.shape[1]))

In [ ]:
print("There are {} observations and {} features in this test dataset. \n".format(reg_test.shape[0],reg_test.shape[1]))

## Regression Classification

Regression analysis is a set of statistical processes for estimating the relationships between a dependent variable (often called the 'outcome variable', or 'target') and one or more independent variables (often called 'features', 'predictors', or 'covariates'). The objective of regression in machine learning is to predict continuous values such as sales amount, quantity, temperature etc.

In order to demonstrate the predict_model() function on unseen data, a sample of records has been withheld from the original dataset to be used for predictions. This should not be confused with a train/test split as this particular split is performed to simulate a real life scenario. Another way to think about this is that these records are not available at the time when the machine learning experiment was performed.

In [ ]:
data_unseen = reg_test
#data.reset_index(inplace=True, drop=True)
#data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(reg_train.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

## 1.0 Setting up environment in PyCaret

The setup() function initializes the environment in pycaret and creates the transformation pipeline to prepare the data for modeling and deployment. setup() must be called before executing any other function in pycaret. It takes two mandatory parameters: a pandas dataframe and the name of the target column. All other parameters are optional and are used to customize the pre-processing pipeline.

When setup() is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties. The data type should be inferred correctly but this is not always the case. To account for this, PyCaret displays a table containing the features and their inferred data types after setup() is executed. If all of the data types are correctly identified enter can be pressed to continue or quit can be typed to end the expriment. Ensuring that the data types are correct is of fundamental importance in PyCaret as it automatically performs a few pre-processing tasks which are imperative to any machine learning experiment. These tasks are performed differently for each data type which means it is very important for them to be correctly configured.

In [ ]:
start_time = datetime.now() # Set start point for time analysis

exp_clf101 = setup(data = reg_train, target = 'Target', session_id=123, numeric_features = categorical_cols+numerical_cols)

end_time = datetime.now() # Set end point for time analysis
print('Duration: {}'.format(end_time - start_time)) # Print out anlysed time

## 2.0 Comparing all models

Comparing all models to evaluate performance is the recommended starting point for modeling once the setup is completed (unless you exactly know what kind of model you need, which is often not the case). This function trains all models in the model library and scores them using k-fold cross validation for metric evaluation. The output prints a score grid that shows average MAE, MSE, RMSE, R2, RMSLE and MAPE accross the folds (10 by default) along with training time.

Run if you need to compare only!

In [ ]:
#start_time = datetime.now()

#best_model = compare_models()

#end_time = datetime.now()
#print('Duration: {}'.format(end_time - start_time))

Two simple words of code (not even a line) have trained and evaluated over 20 models using cross validation. The score grid printed above highlights the highest performing metric for comparison purposes only. The grid by default is sorted using R2 (highest to lowest) which can be changed by passing sort parameter. For example compare_models(sort = 'RMSLE') will sort the grid by RMSLE (lower to higher since lower is better). If you want to change the fold parameter from the default value of 10 to a different value then you can use the fold parameter. For example compare_models(fold = 5) will compare all models on 5 fold cross validation. Reducing the number of folds will improve the training time. By default, compare_models return the best performing model based on default sort order but can be used to return a list of top N models by using n_select parameter. </br>

Notice that how exclude parameter is used to block certain models (in this case RANSAC).

In [ ]:
#print(best_model)

## 3.0 Create a Model

create_model is the most granular function in PyCaret and is often the foundation behind most of the PyCaret functionalities. As the name suggests this function trains and evaluates a model using cross validation that can be set with fold parameter. The output prints a score grid that shows MAE, MSE, RMSE, R2, RMSLE and MAPE by fold.

There are 25 regressors available in the model library of PyCaret. To see list of all regressors either check the docstring or use models function to see the library.

### 3.1 AdaBoost Regressor

In [ ]:
#ada = create_model('ada')

In [ ]:
#print(ada)

### 3.2 Light Gradient Boosting Machine

In [ ]:
lightgbm = create_model('lightgbm')

Notice that the Mean score of all models matches with the score printed in compare_models(). This is because the metrics printed in the compare_models() score grid are the average scores across all CV folds. Similar to compare_models(), if you want to change the fold parameter from the default value of 10 to a different value then you can use the fold parameter. For Example: create_model('dt', fold = 5) to create Decision Tree using 5 fold cross validation.

### ...

## 4. Tune a Model

When a model is created using the create_model function it uses the default hyperparameters to train the model. In order to tune hyperparameters, the tune_model function is used. This function automatically tunes the hyperparameters of a model using Random Grid Search on a pre-defined search space. The output prints a score grid that shows MAE, MSE, RMSE, R2, RMSLE and MAPE by fold. To use the custom search grid, you can pass custom_grid parameter in the tune_model function (see 4.2 LightGBM tuning below).

In [ ]:
tuned_lightgbm=tune_model(lightgbm, optimize = 'RMSE')

### 4.1 AdaBoost Regressor

In [ ]:
#tuned_ada = tune_model(ada)

In [ ]:
#print(tuned_ada)

### 4.2 Light Gradient Boosting Machine

In [ ]:
# lgbm_params = {
#     'learning_rate': [0.005,0.01,0.015],   
#     'n_estimators': [40,100,200],          
#     'num_leaves': [6,8,12,15,16],          
#     'boosting_type' : ['gbdt'],
#     'objective' : ['regression'],          
#     'metric' : ['rmse'],                   
#     'colsample_bytree' : [0.6, 0.8, 1],    
#     'subsample' : [0.7,0.9, 1],            
#     'reg_alpha' : [0,1],                   
#     'reg_lambda' : [0,1],                  
#     'max_leaves': [128,256,512],           
#     'min_child_samples' : [1,20]           
#             }

In [ ]:
# tuned_lightgbm = tune_model(lightgbm, custom_grid = lgbm_params)

In [ ]:
print(tuned_lightgbm)

By default, tune_model optimizes R2 but this can be changed using optimize parameter. For example: tune_model(dt, optimize = 'MAE') will search for the hyperparameters of a Decision Tree Regressor that results in the lowest MAE instead of highest R2. For the purposes of this example, we have used the default metric R2 for the sake of simplicity only.

Metrics alone are not the only criteria you should consider when finalizing the best model for production. Other factors to consider include training time, standard deviation of k-folds etc.

### ...

## 5. Plot a Model

Before model finalization, the plot_model() function can be used to analyze the performance across different aspects such as Residuals Plot, Prediction Error, Feature Importance etc. This function takes a trained model object and returns a plot based on the test / hold-out set.

There are over 10 plots available, please see the plot_model() docstring for the list of available plots.

### 5.1 Residual Plot

In [ ]:
plot_model(lightgbm)

### 5.2 Prediction Error Plot

In [ ]:
plot_model(lightgbm, plot = 'error')

### 5.3 Feature Importance Plot

In [ ]:
plot_model(lightgbm, plot='feature')

Another way to analyze the performance of models is to use the evaluate_model() function which displays a user interface for all of the available plots for a given model. It internally uses the plot_model() function.

In [ ]:
evaluate_model(lightgbm)

## 6 Predict on test / hold-out Sample

Before finalizing the model, it is advisable to perform one final check by predicting the test/hold-out set and reviewing the evaluation metrics. If you look at the information grid in Section 1 above, you will see that 100% (xxxx samples) of the data has been separated out as a test/hold-out sample. Now, using our final trained model stored in the tuned_lightgbm variable we will predict the hold-out sample and evaluate the metrics to see if they are materially different than the CV results.

In [ ]:
lightgbm = ensemble_model(lightgbm, method = 'Bagging')

In [ ]:
predict_model(lightgbm);

The R2 on the test/hold-out set is 0.xxxx compared to 0.xxxx achieved on tuned_lightgbm CV results. This is not a significant difference. If there is a large variation between the test/hold-out and CV results, then this would normally indicate over-fitting but could also be due to several other factors and would require further investigation. In this case, we will move forward with finalizing the model and predicting on unseen data.

(TIP : It's always good to look at the standard deviation of CV results when using create_model.)

## 7 Finalize Model for Deplyoment

Model finalization is the last step in the experiment. A normal machine learning workflow in PyCaret starts with setup(), followed by comparing all models using compare_models() and shortlisting a few candidate models (based on the metric of interest) to perform several modeling techniques such as hyperparameter tuning, ensembling, stacking etc. This workflow will eventually lead you to the best model for use in making predictions on new and unseen data. The finalize_model() function fits the model onto the complete dataset including the test/hold-out sample (100% in this case). The purpose of this function is to train the model on the complete dataset before it is deployed in production.

In [ ]:
final_lightgbm = finalize_model(lightgbm)

In [ ]:
print(final_lightgbm)

Caution: One final word of caution. Once the model is finalized using finalize_model(), the entire dataset including the test/hold-out set is used for training. As such, if the model is used for predictions on the hold-out set after finalize_model() is used, the information grid printed will be misleading as you are trying to predict on the same data that was used for modeling. In order to demonstrate this point only, we will use final_lightgbm under predict_model() to compare the information grid with the one above in section 11.

In [ ]:
predict_model(final_lightgbm); 

Notice how the R2 in the final_lightgbm has increased to 0.xxxx from 0.xxxx, even though the model is same. This is because the final_lightgbm variable is trained on the complete dataset including the test/hold-out set.

## 8. Predict on unseen data

The predict_model() function is also used to predict on the unseen dataset. The only difference from section 11 above is that this time we will pass the data_unseen parameter. data_unseen is the variable created at the beginning.

In [ ]:
unseen_predictions = predict_model(final_lightgbm, data=data_unseen)
#unseen_predictions.head()

The Label and Score columns are added onto the data_unseen set. Label is the prediction and score is the probability of the prediction. Notice that predicted results are concatenated to the original dataset while all the transformations are automatically performed in the background. You can also check the metrics on this since you have actual target column default available. To do that we will use pycaret.utils module. See example below:

In [ ]:
check_metric(unseen_predictions["Target"], unseen_predictions.Label, 'RMSE')

## 9. Saving the model

We have now finished the experiment by finalizing the tuned_rf model which is now stored in final_rf variable. We have also used the model stored in final_rf to predict data_unseen. This brings us to the end of our experiment, but one question is still to be asked: What happens when you have more new data to predict? Do you have to go through the entire experiment again? The answer is no, PyCaret's inbuilt function save_model() allows you to save the model along with entire transformation pipeline for later use.

In [ ]:
save_model(final_lightgbm,'models/Reg_lgbm_Model_{}'.format(d1))

## 10. Loading the saved model

To load a saved model at a future date in the same or an alternative environment, we would use PyCaret's load_model() function and then easily apply the saved model on new unseen data for prediction.

In [ ]:
saved_final_lightgbm = load_model('models/Reg_lgbm_Model_{}'.format(d1))

## 11. Creating submission file

### Saving the Labels for regression labels

In [ ]:
sub_reg = unseen_predictions['Label']
sub_reg.head()

In [ ]:
sub_reg.to_csv("models/sub_reg.csv",index=False)

### Calling back reg and class models

In [ ]:
sub_reg = pd.read_csv("models/sub_reg.csv")
sub_class = pd.read_csv("models/sub_class.csv")

In [ ]:
pred_test =  sub_reg.Label 

### Getting FullvisitorID for aggregating with respect to it

In [ ]:
test_id = dataset_test["fullVisitorId"].values
pred_target = pd.DataFrame({"fullVisitorId":test_id})

### This is the part to get Predict Target with fullVisitorId

In [ ]:
# Removing negative values
pred_test[pred_test<0] = 0
pred_target["PredictedLogRevenue"] = np.expm1(pred_test)
pred_target = pred_target.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()

### Submission file ready below !

In [ ]:
pred_target.columns = ["fullVisitorId", "PredictedLogRevenue"]
pred_target["PredictedLogRevenue"] = np.log1p(pred_target["PredictedLogRevenue"])
pred_target.to_csv("models/submission.csv", index=False)

In [ ]:
# Final submission file
pred_target.head()

## 12. RMSE Test manually

### This is the part to get Actual Target with fullVisitorId to compare the previously Predict Target

In [ ]:
act_target = pd.DataFrame({"fullVisitorId":test_id})
act_target.head()

In [ ]:
act_target["Target_actual"] = np.expm1(unseen_predictions['Target'])
act_target = act_target.groupby("fullVisitorId")["Target_actual"].sum().reset_index()

In [ ]:
act_target.columns = ["fullVisitorId", "Target_actual"]
act_target["Target_actual"] = np.log1p(1+act_target["Target_actual"])
act_target.head()

In [ ]:
RMSE = np.sqrt((sum( (act_target['Target_actual'].values - pred_target['PredictedLogRevenue'].values) **2)) / len(act_target))

In [ ]:
print(f'Your RMSE Score is: {RMSE}')